# SDXL-Turbo: Visualizing the embedding space with Parti Prompts V2 using TensorBoard Projector

In [5]:
import torch
from evolutionary_prompt_embedding.image_creation import SDXLPromptEmbeddingImageCreator
from datasets import load_dataset

In [6]:
creator = SDXLPromptEmbeddingImageCreator(batch_size=1, inference_steps=1)

Keyword arguments {'safety_checker': None, 'requires_safety_checker': False} are not expected by StableDiffusionXLPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loaded StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.31.0",
  "_name_or_path": "stabilityai/sdxl-turbo",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerAncestralDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}



## Using the embedding projector
Online Version: https://projector.tensorflow.org/

In [7]:
from datasets import load_dataset
from typing import List
from evolutionary_prompt_embedding.argument_types import PooledPromptEmbedData

# Load dataset and extract prompts and categories.
prompts_dataset = load_dataset("nateraw/parti-prompts")
prompts = prompts_dataset["train"]["Prompt"]
categories = prompts_dataset["train"]["Category"]
embeddings = [creator.arguments_from_prompt(p) for p in prompts]

# Define additional artist names
additional_prompts = ["Pablo Picasso", "Vincent van Gogh", "Leonardo da Vinci", "Frida Kahlo", "Claude Monet"]
additional_categories = ["Custom"] * len(additional_prompts)
additional_embeddings = [creator.arguments_from_prompt(p) for p in additional_prompts]

# Add the additional entries to the lists
prompts.extend(additional_prompts)
categories.extend(additional_categories)
embeddings.extend(additional_embeddings)

#testlength = 3000 # See how many embeddings are supported (2GB Limit is around 3k for all types)
# Repeat the lists until they reach the desired length
#prompts = (prompts * (testlength // len(prompts) + 1))[:testlength]
#categories = (categories * (testlength // len(categories) + 1))[:testlength]
#embeddings = (embeddings * (testlength // len(embeddings) + 1))[:testlength]

Token indices sequence length is longer than the specified maximum sequence length for this model (84 > 77). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (84 > 77). Running this sequence through the model will result in indexing errors


The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['as a beacon over rolling blue hills']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['as a beacon over rolling blue hills']


In [8]:
from evolutionary_prompt_embedding.tensorboard_embed_visualizer import TensorboardEmbedVisualizer, EmbeddingVariant

visualizer = TensorboardEmbedVisualizer[PooledPromptEmbedData, List[str]](["Prompt", "Category"], "vis")
visualizer.add_embeddings(
    ((emb, [p, c]) for emb, p, c in zip(embeddings, prompts, categories))
)
visualizer.generate_visualization(
    sprite_single_image_dim=(64, 64),
)

Including default embeddings.
Including pooled embeddings.
Including combined average embeddings.
Including combined append embeddings.
Saving checkpoint
Checkpoint and metadata saved in vis
Run 'tensorboard --logdir=vis' to visualize your embeddings.


In [ ]:
!tensorboard --logdir=vis

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.18.0 at http://localhost:6007/ (Press CTRL+C to quit)


## Pre-Stages of the Visualizer:

In [ ]:
import os
import torch
import tensorflow as tf
from datasets import load_dataset
from tensorboard.plugins import projector

# Load dataset and extract prompts and categories.
prompts_dataset = load_dataset("nateraw/parti-prompts")
prompts = prompts_dataset["train"]["Prompt"]
categories = prompts_dataset["train"]["Category"]

# Generate embeddings using your creator function.
embeddings = [creator.arguments_from_prompt(p) for p in prompts]
prompt_embeddings = [e.prompt_embeds for e in embeddings]
pooled_prompt_embeddings = [e.pooled_prompt_embeds for e in embeddings]

In [ ]:
# Create four variants:
# 1. Normal: Flatten token embeddings.
# 2. Pooled: Flatten pooled embeddings.
# 3. Combined Average: Average token embeddings → [1,2048] then concat with pooled ([1,1280]) → [1,3328]
# 4. Combined Append: Flatten token embeddings then append flattened pooled embedding.
normal_variant, pooled_variant = [], []
combined_avg_variant, combined_append_variant = [], []

for p_embed, pooled_embed in zip(prompt_embeddings, pooled_prompt_embeddings):
    normal_flat = p_embed.flatten()             # [77*2048]
    pooled_flat = pooled_embed.flatten()          # [1280]
    avg_token = p_embed.mean(dim=1)               # [1,2048]
    combined_avg = torch.cat([avg_token, pooled_embed], dim=-1).flatten()  # [3328]
    combined_append = torch.cat([normal_flat, pooled_flat], dim=0)         # [77*2048 + 1280]

    normal_variant.append(normal_flat)
    pooled_variant.append(pooled_flat)
    combined_avg_variant.append(combined_avg)
    combined_append_variant.append(combined_append)

normal_tensor = torch.stack(normal_variant, dim=0)
pooled_tensor = torch.stack(pooled_variant, dim=0)
combined_avg_tensor = torch.stack(combined_avg_variant, dim=0)
combined_append_tensor = torch.stack(combined_append_variant, dim=0)

In [ ]:
# Convert to NumPy array and create TensorFlow variable for each embedding variant
embedding_normal_tf = tf.Variable(normal_tensor.cpu().numpy(), name="embedding_normal")
embedding_pooled_tf = tf.Variable(pooled_tensor.cpu().numpy(), name="embedding_pooled")
embedding_combined_avg_tf = tf.Variable(combined_avg_tensor.cpu().numpy(), name="embedding_combined_avg")
embedding_combined_append_tf = tf.Variable(combined_append_tensor.cpu().numpy(), name="embedding_combined_append")

In [ ]:
# Set up output folder.
output_folder = "vis"
os.makedirs(output_folder, exist_ok=True)

# Save a checkpoint containing all embeddings.
checkpoint = tf.train.Checkpoint(
    embedding_normal=embedding_normal_tf,
    embedding_pooled=embedding_pooled_tf,
    embedding_combined_avg=embedding_combined_avg_tf,
    embedding_combined_append=embedding_combined_append_tf
)
checkpoint_path = os.path.join(output_folder, "embedding.ckpt")
checkpoint.save(checkpoint_path)

# Save metadata.
metadata_file = os.path.join(output_folder, "metadata.tsv")
with open(metadata_file, "w") as f_meta:
    f_meta.write("Prompt\tCategory\n")
    for prompt, category in zip(prompts, categories):
        safe_prompt = str(prompt).replace("\t", " ").replace("\n", " ")
        safe_category = str(category).replace("\t", " ").replace("\n", " ")
        f_meta.write(f"{safe_prompt}\t{safe_category}\n")

# Set up projector config with four embeddings.
config = projector.ProjectorConfig()

emb_conf = config.embeddings.add()
emb_conf.tensor_name = "embedding_normal/.ATTRIBUTES/VARIABLE_VALUE"
emb_conf.metadata_path = "metadata.tsv"
emb_conf.sprite.image_path = "sprite.png"
emb_conf.sprite.single_image_dim.extend([30, 30])

emb_conf = config.embeddings.add()
emb_conf.tensor_name = "embedding_pooled/.ATTRIBUTES/VARIABLE_VALUE"
emb_conf.metadata_path = "metadata.tsv"
emb_conf.sprite.image_path = "sprite.png"
emb_conf.sprite.single_image_dim.extend([30, 30])

emb_conf = config.embeddings.add()
emb_conf.tensor_name = "embedding_combined_avg/.ATTRIBUTES/VARIABLE_VALUE"
emb_conf.metadata_path = "metadata.tsv"
emb_conf.sprite.image_path = "sprite.png"
emb_conf.sprite.single_image_dim.extend([30, 30])

emb_conf = config.embeddings.add()
emb_conf.tensor_name = "embedding_combined_append/.ATTRIBUTES/VARIABLE_VALUE"
emb_conf.metadata_path = "metadata.tsv"
emb_conf.sprite.image_path = "sprite.png"
emb_conf.sprite.single_image_dim.extend([30, 30])

projector.visualize_embeddings(output_folder, config)
print("Checkpoint and projector config saved in", output_folder)
print("Run 'tensorboard --logdir={}' to visualize your embeddings.".format(output_folder))

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=vis

In [ ]:
# Convenience function to kill the tensorboard process
!kill XXXX